In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,2022-01-29,1,0.0
1,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2022-01-30,1,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,1.0,2.0,0.0,0.0,0.0,0.0,2022-01-31,1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2.0,1.0,0.0,1.0,1.0,0.0,0.0,2022-02-01,1,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-02-02,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81085,6.0,7.0,7.0,4.0,9.0,1.0,3.0,5.0,4.0,7.0,...,5.0,8.0,7.0,2.0,1.0,5.0,1.0,2022-11-26,265,2.0
81086,10.0,5.0,4.0,11.0,8.0,2.0,2.0,5.0,7.0,8.0,...,6.0,2.0,1.0,3.0,2.0,8.0,4.0,2022-11-27,265,3.0
81087,6.0,2.0,2.0,0.0,3.0,7.0,24.0,20.0,10.0,18.0,...,5.0,5.0,4.0,4.0,8.0,1.0,7.0,2022-11-28,265,10.0
81088,4.0,6.0,0.0,1.0,1.0,7.0,22.0,18.0,25.0,17.0,...,10.0,11.0,13.0,3.0,4.0,3.0,1.0,2022-11-29,265,3.0


In [3]:
from datetime import datetime
from src.data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2022, 6, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(32595, 674)
y_train.shape=(32595,)
X_test.shape=(48495, 674)
y_test.shape=(48495,)


In [5]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
       
    tss = TimeSeriesSplit(n_splits=2)
    scores = []
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [6]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

[I 2023-03-30 15:16:28,812] A new study created in memory with name: no-name-d39570d8-97fb-49a1-b579-a57478acd669


[LightGBM] [Warning] bagging_fraction is set=0.9610080178483569, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9610080178483569
[LightGBM] [Warning] feature_fraction is set=0.3181670706394135, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3181670706394135
[LightGBM] [Warning] bagging_fraction is set=0.9610080178483569, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9610080178483569
[LightGBM] [Warning] feature_fraction is set=0.3181670706394135, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3181670706394135


[I 2023-03-30 15:16:47,028] Trial 0 finished with value: 3.4080006903578592 and parameters: {'num_leaves': 30, 'feature_fraction': 0.3181670706394135, 'bagging_fraction': 0.9610080178483569, 'min_child_samples': 61}. Best is trial 0 with value: 3.4080006903578592.


[LightGBM] [Warning] bagging_fraction is set=0.25717465235111003, subsample=1.0 will be ignored. Current value: bagging_fraction=0.25717465235111003
[LightGBM] [Warning] feature_fraction is set=0.5016908756461534, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5016908756461534
[LightGBM] [Warning] bagging_fraction is set=0.25717465235111003, subsample=1.0 will be ignored. Current value: bagging_fraction=0.25717465235111003
[LightGBM] [Warning] feature_fraction is set=0.5016908756461534, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5016908756461534


[I 2023-03-30 15:17:28,685] Trial 1 finished with value: 3.3020680098819817 and parameters: {'num_leaves': 88, 'feature_fraction': 0.5016908756461534, 'bagging_fraction': 0.25717465235111003, 'min_child_samples': 9}. Best is trial 1 with value: 3.3020680098819817.


...

[I 2023-03-30 15:17:46,071] Trial 2 finished with value: 3.7509612089050552 and parameters: {'num_leaves': 26, 'feature_fraction': 0.48095128392287717, 'bagging_fraction': 0.9124975920165295, 'min_child_samples': 95}. Best is trial 1 with value: 3.3020680098819817.


[LightGBM] [Warning] bagging_fraction is set=0.6891037727787297, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6891037727787297
[LightGBM] [Warning] feature_fraction is set=0.37395244876722605, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.37395244876722605
[LightGBM] [Warning] bagging_fraction is set=0.6891037727787297, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6891037727787297
[LightGBM] [Warning] feature_fraction is set=0.37395244876722605, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.37395244876722605


[I 2023-03-30 15:18:18,430] Trial 3 finished with value: 3.434745412438007 and parameters: {'num_leaves': 117, 'feature_fraction': 0.37395244876722605, 'bagging_fraction': 0.6891037727787297, 'min_child_samples': 23}. Best is trial 1 with value: 3.3020680098819817.


[LightGBM] [Warning] bagging_fraction is set=0.3183905772306232, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3183905772306232
[LightGBM] [Warning] feature_fraction is set=0.809126944524821, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.809126944524821
[LightGBM] [Warning] bagging_fraction is set=0.3183905772306232, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3183905772306232
[LightGBM] [Warning] feature_fraction is set=0.809126944524821, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.809126944524821


[I 2023-03-30 15:19:03,567] Trial 4 finished with value: 3.8132667121994523 and parameters: {'num_leaves': 215, 'feature_fraction': 0.809126944524821, 'bagging_fraction': 0.3183905772306232, 'min_child_samples': 100}. Best is trial 1 with value: 3.3020680098819817.


In [7]:
# Extract best params from optuna
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 88, 'feature_fraction': 0.5016908756461534, 'bagging_fraction': 0.25717465235111003, 'min_child_samples': 9}


In [10]:
# Retrain the model 

pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

[LightGBM] [Warning] bagging_fraction is set=0.25717465235111003, subsample=1.0 will be ignored. Current value: bagging_fraction=0.25717465235111003
[LightGBM] [Warning] feature_fraction is set=0.5016908756461534, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5016908756461534


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x7fa036e2ee50>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.25717465235111003,
                               feature_fraction=0.5016908756461534,
                               min_child_samples=9, num_leaves=88))])

In [13]:
# Compute the test error

predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=2.5662


In [14]:
from src.plot import plot_one_sample

plot_one_sample(
    features=X_test,
    targets=y_test,
    example_id=2979,
    predictions=pd.Series(predictions)
)

In [15]:
plot_one_sample(
    features=X_test,
    targets=y_test,
    example_id=3979,
    predictions=pd.Series(predictions)
)